In [1]:
# This information helps with debugging and getting support :)
import sys, platform
import pandas as pd
import bifacial_radiance as br
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("bifacial_radiance version ", br.__version__)

Working on a  Windows 10
Python version  3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]
Pandas version  1.5.3
bifacial_radiance version  0.4.3.dev341+g93e0ec5.d20240830


# 21 - Weather to Module Performance
## Modeling Performance, an End to End Simulation

This tutorial shows how to use the new function on bifacial_radiance calculatePerformanceModule performance, as well as how to find CEC Module parameters.


In [2]:
import os
from pathlib import Path

testfolder = str(Path().resolve().parent.parent / 'bifacial_radiance' / 'TEMP' / 'Tutorial_21')

if not os.path.exists(testfolder): os.mkdir(testfolder)

# Another option using relative address; for some operative systems you might need '/' instead of '\'
# testfolder = os.path.abspath(r'..\..\bifacial_radiance\TEMP')  

print ("Your simulation will be stored in %s" % testfolder)


Your simulation will be stored in C:\Users\cdeline\Documents\Python Scripts\Bifacial_Radiance\bifacial_radiance\TEMP\Tutorial_21


In [3]:
import bifacial_radiance
import numpy as np
import pandas as pd
import pvlib 

bifacial_radiance.__version__

'0.4.3.dev341+g93e0ec5.d20240830'

## Geting a CEC Module to pass into demo.makeModule

In [4]:
url = 'https://raw.githubusercontent.com/NREL/SAM/patch/deploy/libraries/CEC%20Modules.csv'
db = pd.read_csv(url, index_col=0) # Reading this might take 1 min or so, the database is big.

Find the module that you want. In this case we know it's a SunPower of model SPR-E19-310-COM. 

Make sure you select only 1 module from the database -- sometimes there are similar names.

In [5]:
modfilter2 = db.index.str.startswith('SunPower') & db.index.str.endswith('SPR-E19-310-COM')
print(modfilter2)
CECMod = db[modfilter2]
print(len(CECMod), " modules selected. Name of 1st entry: ", CECMod.index[0])

[False False False ... False False False]
1  modules selected. Name of 1st entry:  SunPower SPR-E19-310-COM


In [6]:
# Selecting only two times as examples
starttime = '01_13_11';  endtime = '01_13_12'
demo = bifacial_radiance.RadianceObj('tutorial_21', path = testfolder) # Create a RadianceObj 'object'
weatherfile = demo.getEPW(lat = 37.5, lon = -77.6)  # This location corresponds to Richmond, VA.
metdata = demo.readWeatherFile(weatherFile=weatherfile, starttime=starttime, endtime=endtime)
demo.setGround(0.2)

path = C:\Users\cdeline\Documents\Python Scripts\Bifacial_Radiance\bifacial_radiance\TEMP\Tutorial_21
Getting weather file: USA_VA_Richmond.724010_TMY2.epw
 ... OK!
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Filtering dates
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos
Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.


The CEC data should be passed into the ModuleObj, either at time of creation, or sometime before it is passed into makeScene.

In [7]:
mymodule = demo.makeModule(name='test-module', x=1, y=2, bifi=0.9, CECMod=CECMod) 


Module Name: test-module
Module test-module updated in module.json
Pre-existing .rad file objects\test-module.rad will be overwritten



The same data could instead be passed after the ModuleObj's definition, or at time of performance analysis:

In [8]:
mymodule.addCEC(CECMod)

In [9]:
# Let's make a second module, and set it to the default Prism Solar module type
mymodule2 = demo.makeModule(name='test', x=1, y=2, bifi=0.8, CECMod=None) 


Module Name: test
Module test updated in module.json
Pre-existing .rad file objects\test.rad will be overwritten



We're going to set up two scenes, each with a different module type!

In [10]:
sceneDict = {'tilt': 0, 'azimuth': 180, 'pitch': 5,'hub_height':1.5, 'nMods':5, 'nRows': 2}
sceneDict2 = {'tilt': 0, 'azimuth': 180, 'pitch': 5,'hub_height':2.5, 'nMods':2, 'nRows': 1, 'originx': -15}
trackerdict = demo.set1axis(metdata = metdata, cumulativesky = False)
trackerdict = demo.gendaylit1axis()
trackerdict = demo.makeScene1axis(trackerdict, module = mymodule, sceneDict = sceneDict)

Creating ~2 skyfiles. 
Created 2 skyfiles in /skies/

Making ~2 .rad files for gendaylit 1-axis workflow (this takes a minute..)
2 Radfiles created in /objects/


Make a second scene with the other module type

In [11]:
trackerdict = demo.makeScene1axis(trackerdict, module = mymodule2, sceneDict=sceneDict2, append=True)
trackerdict = demo.makeOct1axis()
trackerdict = demo.analysis1axis(sensorsy=3)
trackerdict = demo.analysis1axis(sensorsy=3, sceneNum=1)


Making ~2 .rad files for gendaylit 1-axis workflow (this takes a minute..)
2 Radfiles created in /objects/

Making 2 octfiles in root directory.
Created 1axis_2021-01-13_1100.oct
Created 1axis_2021-01-13_1200.oct
Linescan in process: 1axis_2021-01-13_1100_Scene0_Row1_Module3_Front
Linescan in process: 1axis_2021-01-13_1100_Scene0_Row1_Module3_Back
Saved: results\irr_1axis_2021-01-13_1100_Scene0_Row1_Module3.csv
Index: 2021-01-13_1100. Wm2Front: 254.07483333333334. Wm2Back: 39.88008333333333
Linescan in process: 1axis_2021-01-13_1200_Scene0_Row1_Module3_Front
Linescan in process: 1axis_2021-01-13_1200_Scene0_Row1_Module3_Back
Saved: results\irr_1axis_2021-01-13_1200_Scene0_Row1_Module3.csv
Index: 2021-01-13_1200. Wm2Front: 253.1335. Wm2Back: 37.80824666666667
Linescan in process: 1axis_2021-01-13_1100_Scene1_Row1_Module1_Front
Linescan in process: 1axis_2021-01-13_1100_Scene1_Row1_Module1_Back
Saved: results\irr_1axis_2021-01-13_1100_Scene1_Row1_Module1.csv
Index: 2021-01-13_1100. Wm2F

## Calculating the Performance and Exporting the Results to a CSV

In [12]:
#print(trackerdict)
#tracker_dict_sample = {'2021-01-13_1100':trackerdict['2021-01-13_1100']}
#eff_irr = tracker_dict_sample['Wm2Front'] + tracker_dict_sample['Wm2Back']
Compiled_Results = demo.calculatePerformance1axis()
print(Compiled_Results)


No CECModule data passed; using default for Prism Solar BHC72-400
         timestamp                          name modNum rowNum sceneNum  \
0  2021-01-13_1100  1axis_2021-01-13_1100_Scene0      3      1        0   
1  2021-01-13_1100  1axis_2021-01-13_1100_Scene1      1      1        1   
2  2021-01-13_1200  1axis_2021-01-13_1200_Scene0      3      1        0   
3  2021-01-13_1200  1axis_2021-01-13_1200_Scene1      1      1        1   

                                     x  \
0  [0.3734448, 0.01462469, -0.3441954]   
1     [-14.62656, -14.98538, -15.3442]   
2  [0.473756, 0.007594116, -0.4585678]   
3    [-14.52624, -14.99241, -15.45857]   

                                             y  \
0   [2.28669e-17, 8.955042e-19, -2.107589e-17]   
1   [2.28669e-17, 8.955042e-19, -2.107589e-17]   
2  [2.900919e-17, 4.650055e-19, -2.807918e-17]   
3  [2.900919e-17, 4.650055e-19, -2.807918e-17]   

                                z  \
0   [1.166863, 1.51507, 1.863277]   
1   [2.166863, 2.51507

In [13]:
demo.exportTrackerDict(savefile=os.path.join('results','Final_Results.csv'),reindex=False)
pd.read_csv(os.path.join('results','Final_Results.csv'))

Exporting TrackerDict
3
3
1
1


,Unnamed: 0,timestamp,rowNum,modNum,sceneNum,name,Wm2Front,Wm2Back,DNI,DHI,...,theta,surf_tilt,surf_azm,POA_eff,Gfront_mean,Grear_mean,Pout_raw,Mismatch,Pout,Pout_Gfront
0,0,2021-01-13_1100,1,3,0,1axis_2021-01-13_1100_Scene0,[253.4203 254.3263 254.4779],[39.27898 39.55687 40.8044 ],144,149,...,-44.14,44.14,90.0,289.966908,254.074833,39.880083,91.687503,0.031298,91.658807,80.249342
1,1,2021-01-13_1100,1,1,1,1axis_2021-01-13_1100_Scene1,[253.0821 253.8797 253.6734],[52.66748 53.1945 52.93923],144,149,...,-44.14,44.14,90.0,295.892056,253.545067,52.933737,123.748694,0.009121,123.737407,106.070184
2,2,2021-01-13_1200,1,3,0,1axis_2021-01-13_1200_Scene0,[253.0991 253.2366 253.0648],[37.59769 37.44495 38.3821 ],97,200,...,-21.20,21.20,90.0,287.160922,253.133500,37.808247,90.128038,0.019082,90.110840,79.356997
3,3,2021-01-13_1200,1,1,1,1axis_2021-01-13_1200_Scene1,[252.2994 252.4342 252.5692],[50.62335 50.9784 50.82404],97,200,...,-21.20,21.20,90.0,293.081144,252.434267,50.808597,121.759562,0.006176,121.752043,104.892870


## Now look at gencumulativesky tracking workflow

In [14]:
starttime = '01_13_11';  endtime = '12_13_12'
demo = bifacial_radiance.RadianceObj('tutorial_21', path = testfolder) # Create a RadianceObj 'object'
weatherfile = demo.getEPW(lat = 37.5, lon = -77.6)  # This location corresponds to Richmond, VA.
metdata = demo.readWeatherFile(weatherFile=weatherfile, starttime=starttime, endtime=endtime)
#metdata = demo.readWeatherFile(weatherFile=weatherfile)
demo.setGround(0.2)
mymodule = demo.makeModule(name='test-module', x=1, y=2, bifi=0.9, CECMod=CECMod) 

path = C:\Users\cdeline\Documents\Python Scripts\Bifacial_Radiance\bifacial_radiance\TEMP\Tutorial_21
Getting weather file: USA_VA_Richmond.724010_TMY2.epw
 ... OK!
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Filtering dates
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos
Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.

Module Name: test-module
Module test-module updated in module.json
Pre-existing .rad file objects\test-module.rad will be overwritten



In [15]:
sceneDict = {'tilt': 0, 'azimuth': 180, 'pitch': 5,'hub_height':1.5, 'nMods':5, 'nRows': 2}
trackerdict = demo.set1axis(metdata=metdata, cumulativesky=True, limit_angle=15, backtrack=False)
trackerdict = demo.genCumSky1axis()
trackerdict = demo.makeScene1axis(trackerdict, module = mymodule, sceneDict = sceneDict)
trackerdict = demo.makeOct1axis()
trackerdict = demo.analysis1axis(modWanted = [2,4], sensorsy=3)

Saving file EPWs\1axis_-15.0.csv, # points: 1875
Saving file EPWs\1axis_-10.0.csv, # points: 241
Saving file EPWs\1axis_0.0.csv, # points: 2
Saving file EPWs\1axis_5.0.csv, # points: 242
Saving file EPWs\1axis_10.0.csv, # points: 33
Saving file EPWs\1axis_15.0.csv, # points: 1683
message: There were 1833 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_-15.0.rad
message: There were 238 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_-10.0.rad
message: There were 2 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_0.0.rad
message: There were 239 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_5.0.rad
message: There were 33 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_10.0.rad
message: There were 1661 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_15.0.rad

Makin

In [16]:
demo.calculatePerformance1axis() # saves to demo.CompiledResults and results/Cumulative_Results.csv
print(demo.CompiledResults)

   row  module        BGG   Gfront_mean     Grear_mean  \
0    1       2  11.240246  1.669446e+06  208499.878233   
1    1       4  10.355119  1.657496e+06  190706.322967   

                                             POA_eff  \
0  [1858883.9889933334, 1854803.3519133334, 18576...   
1  [1831434.8003633332, 1826145.2824733332, 18298...   

                                             Wm2Back  
0  [210486.20640000002, 205952.16520000002, 20906...  
1  [193265.28869999998, 187388.04659999997, 19146...  


In [17]:
pd.read_csv(os.path.join('results','Cumulative_Results.csv'))

,Unnamed: 0,row,module,BGG,Gfront_mean,Grear_mean,POA_eff,Wm2Back
0,0,1,2,11.240,1669446.403,208499.878,"[1858883.9889933334, 1854803.3519133334, 18576...","[210486.20640000002, 205952.16520000002, 20906..."
1,1,1,4,10.355,1657496.041,190706.323,"[1831434.8003633332, 1826145.2824733332, 18298...","[193265.28869999998, 187388.04659999997, 19146..."
